In [ ]:
import numpy as np
import pandas as pd

# Load dataset

In [ ]:
df = pd.read_csv('../dataset/emails.csv')
df

In [ ]:
df.shape

In [ ]:
df.columns

# Preprocessing data

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
X = df['text']
X

In [ ]:
y = df['spam']
y

 -------------------------------------------------------

# CountVectorizer

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# cv = CountVectorizer()

In [ ]:
# X = cv.fit_transform(X)

In [ ]:
# cv.get_feature_names_out()

In [ ]:
# my_df = pd.DataFrame(data=X.toarray(), columns=cv.get_feature_names_out())

In [ ]:
# my_df['isSpam'] = y.tolist()

In [ ]:
# my_df

In [ ]:
# X = pd.DataFrame(X.toarray()) # konverzija u DataFrame

--------------------------------------------------------------

# TF-IDF matrix

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tf = TfidfVectorizer(max_df=0.8) # ignore terms that appear in more than 80% of the documents

In [ ]:
X = tf.fit_transform(X)

In [ ]:
tf.get_feature_names_out()

In [ ]:
my_df = pd.DataFrame(data=X.toarray(), columns = tf.get_feature_names_out())

In [ ]:
my_df['isSpam'] = y.tolist()

In [ ]:
my_df.columns

In [ ]:
my_df

In [ ]:
X = pd.DataFrame(X.toarray())

----------------------------------------------

In [ ]:
import gensim

df['clean message'] = X.apply(lambda x: gensim.utils.simple_preprocess(x))
df.head()

-------------------------------------------------------

# Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['clean message'], y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

# Word2vector

In [ ]:
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=200,
                                   window=700,
                                   min_count=5)

In [ ]:
words = set(w2v_model.wv.index_to_key )
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train], dtype=object)
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test], dtype=object)

In [ ]:
for i, v in enumerate(X_train_vect):
    print(len(X_train.iloc[i]), len(v))

In [ ]:
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [ ]:
for i, v in enumerate(X_train_vect_avg):
    print(len(X_train.iloc[i]), len(v))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [ ]:
y_pred = rf_model.predict(X_test_vect_avg)

In [ ]:
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test).sum()/len(y_pred), 3)))

-------------------------------------------------------------------

# MLP

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()
model.add(Dense(input_dim=X_train.shape[1], units=100, kernel_regularizer= regularizers.L1(l1=1e-5), activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=64, epochs=40, verbose=1, validation_split=0.2)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
epochs = history.epoch
plt.plot(epochs, history.history['loss']) # blue
plt.plot(epochs, history.history['val_loss']) #orange
plt.title('loss')

In [ ]:
plt.plot(epochs, history.history['accuracy']) #blue
plt.plot(epochs, history.history['val_accuracy']) #orange
plt.title('accuracy')

In [ ]:
model.evaluate(X_test, y_test, batch_size=64)

----------------------------------------------------

# METRICS

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score

In [ ]:
y_pred = model.predict(X_test)

y_pred_new = [int(not(i<0.5)) for i in y_pred]
y_test_new = y_test.to_numpy().tolist()


confusion_matrix(y_test_new, y_pred_new)

In [ ]:
classification_report(y_test_new, y_pred_new)

In [ ]:
precision_score(y_test_new, y_pred_new)